## Task 2, part 1

In [11]:
import pandas as pd
import numpy as np

### 3 functions for Levenshtein, Jaro and Affine similarity

In [12]:
def lsim(s, t):
    """
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """

    rows = len(s)+1
    cols = len(t)+1
    dist = [[0 for x in range(cols)] for x in range(rows)]

    # source prefixes can be transformed into empty strings
    # by deletions:
    for i in range(1, rows):
        dist[i][0] = i

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i

    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                c = 0
            else:
                c = 2
            dist[row][col] = min(dist[row-1][col] + 1,       # deletion
                                 dist[row][col-1] + 1,       # insertion
                                 dist[row-1][col-1] + c)     # substitution

    #for r in range(rows):
     #   print(dist[r])


    med = dist[-1][-1]
    sim = 1 - (med/(max(len(s), len(t))))
    return sim

In [13]:
def jarosim(s, t):
    '''Jaro similarity between two strings.'''
    s_len = len(s)
    t_len = len(t)

    if s_len == 0 and t_len == 0:
        return 1

    match_distance = (max(s_len, t_len) // 2) - 1

    s_matches = [False] * s_len
    t_matches = [False] * t_len

    matches = 0
    transpositions = 0

    for i in range(s_len):
        start = max(0, i - match_distance)
        end = min(i + match_distance + 1, t_len)

        for j in range(start, end):
            if t_matches[j]:
                continue
            if s[i] != t[j]:
                continue
            s_matches[i] = True
            t_matches[j] = True
            matches += 1
            break

    if matches == 0:
        return 0

    k = 0
    for i in range(s_len):
        if not s_matches[i]:
            continue
        while not t_matches[k]:
            k += 1
        if s[i] != t[k]:
            transpositions += 1
        k += 1

    return ((matches / s_len) +
            (matches / t_len) +
            ((matches - transpositions / 2) / matches)) / 3

In [14]:
def affine(s1, s2, match_score=1, mismatch_penalty=-1, gap_open_penalty=-2, gap_extend_penalty=-1):
    # Lengths of the two strings
    len1, len2 = len(s1), len(s2)

    # Initialize score matrices
    M = np.zeros((len1 + 1, len2 + 1))  # Main matrix for matching
    Ix = np.zeros((len1 + 1, len2 + 1))  # Matrix for gaps in string 1
    Iy = np.zeros((len1 + 1, len2 + 1))  # Matrix for gaps in string 2

    # Fill the score matrices with initial gap penalties
    for i in range(1, len1 + 1):
        M[i][0] = gap_open_penalty + (i - 1) * gap_extend_penalty
        Ix[i][0] = gap_open_penalty + (i - 1) * gap_extend_penalty
        Iy[i][0] = -float('inf')

    for j in range(1, len2 + 1):
        M[0][j] = gap_open_penalty + (j - 1) * gap_extend_penalty
        Iy[0][j] = gap_open_penalty + (j - 1) * gap_extend_penalty
        Ix[0][j] = -float('inf')

    # Dynamic programming to fill in the matrices
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            match = M[i - 1][j - 1] + (match_score if s1[i - 1] == s2[j - 1] else mismatch_penalty)
            Ix[i][j] = max(M[i - 1][j] + gap_open_penalty, Ix[i - 1][j] + gap_extend_penalty)
            Iy[i][j] = max(M[i][j - 1] + gap_open_penalty, Iy[i][j - 1] + gap_extend_penalty)
            M[i][j] = max(match, Ix[i][j], Iy[i][j])

    return M[len1][len2]

### Reading the files

In [ ]:
acm = pd.read_csv('./ACM.csv')
dbl = pd.read_csv('./DBLP2.csv', encoding='ISO-8859-1')

acm.head()
dbl.head()

### Function to process and compare two records

In [16]:
def recComp(rec1, rec2): #compares two records

    #remove spaces and make first record lowercase
    rec1_id = rec1['id']
    rec1_title = ' '.join(str(rec1['title']).split()).lower()
    rec1_authors = ' '.join(str(rec1['authors']).split()).lower()
    rec1_venue = ' '.join(str(rec1['venue']).split()).lower()
    rec1_year = rec1['year']

    #removes spaces and make second record lowercase
    rec2_id = rec2['id']
    rec2_title = ' '.join(str(rec2['title']).split()).lower()
    rec2_authors = ' '.join(str(rec2['authors']).split()).lower()
    rec2_venue = ' '.join(str(rec2['venue']).split()).lower()
    rec2_year = rec2['year']

    st = lsim(rec1_title, rec2_title)

    sa = jarosim(rec1_authors, rec2_authors)

    sc_unscaled = affine(rec1_venue, rec2_venue)

    if rec1_year == rec2_year:
      sy = 1
    else:
      sy = 0


    return rec1_id, rec2_id, st, sa, sc_unscaled, sy



### Comparing all records from file 1 with all records from file 2

In [ ]:
k=0
rec_results = pd.DataFrame(columns = ['id1', 'id2', 'st', 'sa', 'sc_unscaled', 'sy'])
for i in range(550):  #(acm.shape[0]): not used, too high runtime
  for j in range(550): #(acm.shape[0]): not used, too high runtime
    rec_results.loc[k] = recComp(acm.iloc[i], dbl.iloc[j])
    k+=1

### Computing (true) positives and precisison

In [ ]:
#apply [0,1] min-max scaling for the unscaled sc values
rec_results['sc'] = (rec_results['sc_unscaled'] - rec_results['sc_unscaled'].min())/(rec_results['sc_unscaled'].max()-rec_results['sc_unscaled'].min())

#compute total similarity score with wi = 1
rec_results['score'] = (rec_results['st'] + rec_results['sa'] + rec_results['sc'] + rec_results['sa'])/4

#empty dataframe with similar records
books = pd.DataFrame()

#fill dataframe with IDs of similar books and sim score
for i in range(len(rec_results)):
  if rec_results.iloc[i]['score'] > 0.7:
    books.loc[i,'id2'] = rec_results.loc[i,'id1']
    books.loc[i,'id1'] = rec_results.loc[i,'id2']
    books.loc[i,'score'] = rec_results.loc[i,'score']

#load data on 
perfect = pd.read_csv('./DBLP-ACM_perfectMapping.csv')


# inner join on record IDs to determine true positives
correct = books.merge(perfect, left_on=['id1', 'id2'], right_on=['idDBLP','idACM'])
print(correct.shape[0]) # true positives
print(books.shape[0]) # found positives
# computing precision with true_post/all_pos
precision = correct.shape[0]/books.shape[0]
print(precision)